# Tabular Model: flat data, without context

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mostly-ai/mostlyai-engine/blob/main/examples/flat.ipynb)

In [ ]:
import pandas as pd
from mostlyai.engine import TabularARGN

# load original data
url = "https://github.com/mostly-ai/public-demo-data/raw/refs/heads/dev/census"
trn_df = pd.read_csv(f"{url}/census.csv.gz")

# create and fit the model
argn = TabularARGN(verbose=1)
argn.fit(trn_df)

# generate synthetic samples
syn_df = argn.sample(n_samples=len(trn_df))

In [ ]:
syn_df.head(5)

### QUALITY ASSURANCE

#### univariate `age`

In [ ]:
print("Original Age:  " + ", ".join([f'q{q*100:.0f}: {trn_df["age"].quantile(q):.0f}' for q in [.1, .25, .5, .75, .9]]))
print("Synthetic Age: " + ", ".join([f'q{q*100:.0f}: {syn_df["age"].quantile(q):.0f}' for q in [.1, .25, .5, .75, .9]]))

#### bivariate `sex` ~ `income`: income gap

In [ ]:
trn_gap = (trn_df[trn_df["sex"] == "Male"]["income"] == ">50K").mean() - (trn_df[trn_df["sex"] == "Female"]["income"] == ">50K").mean()
syn_gap = (syn_df[syn_df["sex"] == "Male"]["income"] == ">50K").mean() - (syn_df[syn_df["sex"] == "Female"]["income"] == ">50K").mean()
print(f"Income Gap {trn_gap:.1%} vs. {syn_gap:.1%}")

#### check consistency between `education` and `education.num`

In [ ]:
pd.crosstab(syn_df["education"], syn_df["education_num"])